In [89]:
import numpy as np

def traceback(val, xval, yval, seq2, seq1):
    sequ1 = ''
    sequ2 = ''
    i = len(seq2)
    j = len(seq1) 

    while (i>0 and j>0):
        if (i>0 and j>0 and val[i][j] == val[i-1][j-1] + (match if seq2[i-1] == seq1[j-1] else mismatch)):
            sequ1 += seq1[j-1]
            sequ2 += seq2[i-1]
            i -= 1
            j -= 1
        elif (i>0 and val[i][j] == xval[i][j]):
            sequ1 += '-'
            sequ2 += seq2[i-1]
            i -= 1
        elif (j>0 and val[i][j] == yval[i][j]):
            sequ1 += seq1[j-1]
            sequ2 += '-'
            j -= 1
    
    while (i>0):
        sequ1 += '-'
        sequ2 += seq2[i-1]
        i -= 1
    while (i>0):
        sequ1 += seq1[j-1]
        sequ2 += '-'
        j -= 1  
    
    sequ1 = sequ1[::-1]
    sequ2 = sequ2[::-1]
    
    return (sequ1, sequ2)

In [90]:
def SW_affine(seq1, seq2):
    rows = len(seq1) + 1 
    cols = len(seq2) + 1

    xval = np.zeros((cols, rows))
    yval = np.zeros((cols, rows))
    val = np.zeros((cols, rows))
    
    for i in range(rows):
        val[i][0] = gap_open + i*gap_extend
        yval[i][0] = -np.inf 
    
    for j in range(cols):
        val[0][j] = gap_open + j*gap_extend
        xval[0][j] = -np.inf  
    
    val[0][0] = 0.
     
    for i in range(1, rows):
        for j in range(1, cols):
            xval[i][j] = max(xval[i-1][j] + gap_extend, val[i-1][j] + gap_open + gap_extend)
            yval[i][j] = max(yval[i][j-1] + gap_extend, val[i][j-1] + gap_open + gap_extend)
            cople = 0.
            if (seq1[j-1] == seq2[i-1]):
                cople = val[i-1][j-1] + match
            else:
                cople = val[i-1][j-1] + mismatch
            
            val[i][j] = max(cople, xval[i][j], yval[i][j])
    return traceback(val, xval, yval, seq2, seq1)

In [91]:
gap_open = -1.
gap_extend = -1
match = 1.
mismatch = -1.
seq1 = 'TCCCAGTTATGTCAGGGGACACGAGCATGCAGAGAC'
seq2 = 'AATTGCCGCCGTCGTTTTCAGCAGTTATGTCAGATC' 
result = SW_affine(seq1, seq2)
print(result[0])
print(result[1])

---T-CC-CAGT-TATGTCAGGGGACACGAGCATG-CAGAGAC
AATTGCCGCCGTCGTTTTCA---G-CA-G-TTATGTCAGA-TC
